In [ ]:
import torch
import torch.nn as nn
from google.colab import files

In [ ]:
train_file = files.upload()

In [ ]:
with open("ptb.train.txt", 'r') as f:
    lines = f.readlines()

In [ ]:
def get_tokens():
  tokens = [list(line) for line in lines]
  return tokens

token = get_tokens()

In [ ]:
def flatten(tokens):
  return [items for i in tokens for items in i]

tokens = flatten(token)
print(len(tokens))

In [ ]:
def unique_char(tokens):
  uniq_tokens = []
  for i in tokens:
    if i not in uniq_tokens:
      uniq_tokens.append(i)
  return uniq_tokens


uniq_tokens = unique_char(tokens)
print(len(uniq_tokens))

In [ ]:
vocab = {}
for e, char in enumerate(uniq_tokens):
  vocab[char] = e

In [ ]:
numerical = [vocab[char] for char in tokens]

In [ ]:
seq_length = 64
num_samples = (len(wiki_numerical) - 1) // seq_length
dataset = wiki_numerical[:num_samples * seq_length].reshape(num_samples, seq_length)
dataset.shape

In [ ]:
batch_size = 32
num_batches = len(dataset) // batch_size
train_iter = dataset[:num_batches * batch_size].reshape((num_batches, batch_size, seq_length))
train_iter.shape

In [ ]:
labels = numerical[1:num_samples * seq_length + 1]).reshape(num_batches, batch_size, seq_length)
labels.shape

In [ ]:
def textify(embedding):
    result = ""
    indices = torch.argmax(embedding, axis=1)
    for idx in indices:
        result += uniq_tokens[int(idx)]
    return result

In [ ]:
print(textify(train_iter[10, :, 3]))
print(textify(labels[10, :, 3]))

----- START OF TRANSFORMER -----

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, sequence_len, d_model, dropout_prob):
        super().__init__()
        self.sequence_len = sequence_len
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout_prob)
        self.register_buffer("positional_encoding", self.get_pos_encoding(d_model, sequence_len), False)

    def get_pos_encoding(self, d_model, max_len):
        encodings = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        two_i = torch.arange(0, d_model, 2, dtype=torch.float32)
        div_term = torch.exp(two_i * -(math.log(10000.0) / d_model))
        encodings[:, 0::2] = torch.sin(position * div_term)
        encodings[:, 1::2] = torch.cos(position * div_term)
        encodings = encodings.unsqueeze(1).requires_grad_(False)
        return encodings

    def forward(self, x):
        pe = self.positional_encoding[:x.shape[0]].detach().requires_grad_(False)
        x = x + pe
        x = self.dropout(x)
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, config):
        pass

    def encode(self, x):
        pass

    def decode(self, x):
        pass

    def forward(self, x):
        pass

In [ ]:
num_epochs = 10
net = Transformer()

optimizer = torch.optim.SGD(net.parameters(), 1e-3)

for epoch in range(num_epochs):
    for x, y in zip(train_iter, labels):
        optimizer.zero_grad()
        y_hat = net(x)
        loss = lossfn(y_hat, y)
        loss.backward()
        optimizer.step()
        